# DPR

In [1]:
!pip install transformers
!pip install flask_ngrok
!pip install pyngrok
!pip install faiss-gpu
!pip install pytorch-lightning
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=6e05d87d8fd158612989291beb12f0c3a2cca65a9bcc8bbac2a3c09028f878d6
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-

In [2]:
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/KPMG/Final"

Mounted at /content/drive


In [3]:
import torch
import types
import transformers
import json
import joblib
import numpy as np
from transformers import AutoModel,AutoTokenizer
import faiss
import os

def load_data(path):
    faiss_index = faiss.read_index(path+"/data/custom_index") # data라는 폴더 안에 아래 4가지 데이터들을 넣어두시면 됩니다
    total_passage = joblib.load(path+"/data/custom_passages")
    total_text = joblib.load(path+"/data/custom_text")
    id_to_document = joblib.load(path+"/data/id_to_document")
    return faiss_index, total_passage, total_text, id_to_document

def search(query, index, total_passage, total_text, id_to_document, DPR, DPR_tokenizer, k=20):
    inputs = DPR_tokenizer(query, padding=True, truncation=True, return_tensors='pt')
    embeddings, _ = DPR(**inputs.to('cuda:0'), return_dict=False)
    query_vector = embeddings[:, 0, :]
    top_k = index.search(query_vector.detach().cpu().numpy(), k)
    passage_ids = top_k[1].tolist()[0]

    output = dict()
    doc_ids = []
    raw_texts = []
    for passage_id in passage_ids:
        doc_ids.append(id_to_document[passage_id])
    for doc_id in doc_ids:
        raw_text = " ".join(total_text[doc_id]['text'])
        if total_text[doc_id].get("name"):
            title = total_text[doc_id].get("name")
        else:
            title = total_text[doc_id].get("titles")[1:-1]
        raw_texts.append((title, raw_text)) # (title, raw_text) 튜플 형태입니다

    output['query'] = query
    output['passage'] = [total_passage[_id] for _id in passage_ids]
    output['article'] = raw_texts
    return output # 딕셔너리 형태로 아웃풋을 보냅니다

In [ ]:
cur_path = path
faiss_index, total_passage, total_text, id_to_document = load_data(cur_path)
DPR = AutoModel.from_pretrained(cur_path+"/my_model_second_q_128_lowlr").to('cuda:0') # 모델은 프로그램이 실행되는 현재 경로에 폴더를 두시면 됩니다.
DPR_tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta-multitask")
DPR.eval()

In [21]:
query = "카카오의 신사업은??" # 질문으로 들어가는 쿼리
output = search(query,faiss_index,total_passage, total_text, id_to_document, DPR, DPR_tokenizer, k=20)
print(output['query'])
print(output['passage'][0])
print("--------------------------------------------------------")
print(output['article'][0])

카카오의 신사업은??
카카오 미래 먹거리 준비 착착 비욘드 전략 속도붙나 [SEP] 해외 사업분야에서는 이지케어텍이 진출한 해외 지역에서 카카오헬스케어의 서비스를 제공한다. 또한 신규시장 공동 개척에도 나선다.아울러 카카오헬스케어가 사업목적을 △인공지능(AI) 기반 의료솔루션 개발과 서비스업 △건강관리서비스업 △정보시스템 종합관리와 유지보수 용역업 △시스템통합구축 서비스 판매업 등으로 명시한 만큼 모바일 기반 디지털 헬스케어 플랫폼을 구축도 추진할 것으로 보인다.
--------------------------------------------------------
("카카오 미래 먹거리 준비 '착착'…'비욘드' 전략 속도붙나", '카카오헬스케어 HIS 기업 이지케어텍에 99억 원 투자카카오엔터테인먼트 최근 1조2000억 원 규모 투자 유치카카오의 미래 전략인 비욘드 모바일과 비욘드 코리아의 성과가 발굴되고 있다. /최문정 기자카카오의 양대 비욘드 전략에 속도가 붙고 있다. 카카오는 비욘드 모바일을 통해 안정적인 신사업 기반을 마련하는 한편, 비욘드 코리아로 국내 시장을 넘어 글로벌 시장에서의 성과를 가시화한다는 목표다. 20일 IT업계에 따르면, 카카오의 자회사 카카오헬스케어는 최근 의료정보시스템(HIS) 전문기업 이지케어텍에 약 99억 원 규모의 투자를 단행했다. 이번 투자는 이지케어텍이 제3자 배정 유상증자 형태로 발행하는 신주를 카카오헬스케어가 전량 인수하는 방식이다. 이번 유상증자로 발행되는 신주는 보통주 44만8776주(주당 2만2060원)로 카카오헬스케어는 약 6.57%의 지분을 확보해 서울대학교병원에 이어 이지케어텍의 2대 주주가 된다.이지케어텍은 지난 2001년 서울대병원 전산팀에서 출발했다. 출범 이후 처방 전달과 약물 안전관리 시스템, 전자의무기록 등 의료용 기록 업무를 솔루션화해 서울대병원과 보라매병원 등의 고객사를 확보했다. 이후 중동, 미국, 일본 등 해외 시장 진출 성과도 냈다. 이번 투자는 황희 카카오헬스케어 대표가 주측이 돼 추진됐

In [22]:
retrieved_passages = search(query,faiss_index,total_passage, total_text, id_to_document, DPR, DPR_tokenizer, k=20)

# T5-QA

In [7]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch.nn.functional as F
from torchmetrics import functional as FM

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast,
    get_linear_schedule_with_warmup,
)

In [8]:
class KPMG_QA_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("paust/pko-t5-base", return_dict=True)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            labels = labels
        )

        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        
        prob = F.softmax(outputs, dim=-1)
        preds = torch.argmax(prob, dim=-1)
        
        acc = FM.accuracy(task="multiclass", num_classes=outputs.shape[-1], preds=preds, target=labels)
        metrics = {"val_acc": acc, "val_loss":loss}
        
        return metrics
    
    def validation_step_end(self, val_step_outputs):
        val_acc = val_step_outputs["val_acc"].cpu()
        val_loss = val_step_outputs["val_loss"].cpu()

        self.log("validation_acc", val_acc, prog_bar=True)
        self.log("validation_loss", val_loss, prog_bar=True)
        
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        
        prob = F.softmax(outputs, dim=-1)
        preds = torch.argmax(prob, dim=-1)
        
        acc = FM.accuracy(task="multiclass", num_classes=outputs.shape[-1], preds=preds, target=labels)
        metrics = {"test_acc": acc, "test_loss":loss}
        
        return metrics
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.00005)

In [9]:
QA_model = KPMG_QA_Model.load_from_checkpoint(path+"/epoch=14-val_loss=0.679.ckpt").to("cuda:0")
QA_tokenizer = T5TokenizerFast.from_pretrained("paust/pko-t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def generate_answer(question):
    source_encoding = QA_tokenizer(
        "question: " + question["query"],
        "context: [CLS]" + " ".join(question["passage"]).replace("[CLS]", ""),
        max_length=2048,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
  ).to("cuda:0")
  
    generated_ids = QA_model.model.generate(
        input_ids = source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=4,
        max_length=8,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
  ).to("cuda:0")

    preds = [
        QA_tokenizer.decode(generated_id, skip_speical_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    ]
    
    return "".join(preds).replace("<pad>","").replace("</s>","")

In [23]:
retrieved_passages["query"]

'카카오의 신사업은??'

In [24]:
generate_answer(retrieved_passages)

'비욘드 모바일'

# Summarization

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

summary_tokenizer = PreTrainedTokenizerFast.from_pretrained('Forturne/KPMG-NARVIS-summarization')
summary_model = BartForConditionalGeneration.from_pretrained('Forturne/KPMG-NARVIS-summarization')
summary_model.to("cuda:0")
summary_model.eval()

In [19]:
def concat_candidate(retrieved_passages):
    answer = generate_answer(retrieved_passages)
    tmp_candidate = []
    s1 = set()
    for i in range(20):
        if answer in retrieved_passages["passage"][i]:
            tmp_candidate.append(i)
    
    if len(tmp_candidate) == 0:
        original_article = " ".join(retrieved_passages["article"][0])
        return original_article
    
    else:
        for j in range(len(tmp_candidate)):
            s1.add(" ".join(retrieved_passages["article"][tmp_candidate[j]]))
        original_article = " ".join(s1)
        return original_article

def summarize(retrieved_passages):
    text = concat_candidate(retrieved_passages)
    
    raw_input_ids = summary_tokenizer.encode(text)
    quotient = len(raw_input_ids) // 512
    rest = len(raw_input_ids) % 512
    
    if quotient > 1:
        L = []
        N = len(raw_input_ids)//quotient
        for i in range(quotient):
            input_ids = [summary_tokenizer.bos_token_id] + raw_input_ids[N*i:N*i + N] + [summary_tokenizer.eos_token_id]
            summary_ids = summary_model.generate(torch.tensor([input_ids]).to("cuda:0"),  num_beams=4,  max_length=512,  eos_token_id=1)
            tmp_summarized_text = summary_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
            L.append(tmp_summarized_text)
        
        input_ids = [summary_tokenizer.bos_token_id] + raw_input_ids[N*quotient:N*quotient+rest] + [summary_tokenizer.eos_token_id]
        summary_ids = summary_model.generate(torch.tensor([input_ids]).to("cuda:0"),  num_beams=4,  max_length=512,  eos_token_id=1)
        tmp_summarized_text = summary_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        L.append(tmp_summarized_text)
        
        return " ".join(L)
    
    else:
        input_ids = [summary_tokenizer.bos_token_id] + raw_input_ids + [summary_tokenizer.eos_token_id]
        summary_ids = summary_model.generate(torch.tensor([input_ids]).to("cuda:0"),  num_beams=4,  max_length=512,  eos_token_id=1)
        summarized_text = summary_tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        return summarized_text

In [25]:
summarize(retrieved_passages)

'20일 IT업계에 따르면, 카카오의 자회사 카카오헬스케어는 최근 의료정보시스템(HIS) 전문기업 이지케어텍에 약 99억 원 규모의 투자를 단행하며 국내 시장을 넘어 글로벌 시장에서의 성과를 가시화한다는 목표와 함께 비욘드 모바일을 통해 안정적인 신사업 기반을 마련하는 한편, 비욘드 코리아로 국내 시장을 넘어 글로벌 시장에서의 성과를 가시화한다는 목표다.'

# Flask

In [27]:
import io
import json
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

In [28]:
!ngrok authtoken 2KRO36NhveZmYSb6Y0UShMHkMBK_3NDDN1wmF15haufztEEo4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [29]:
app = Flask(__name__)
run_with_ngrok(app)
@app.route("/")
def index():
  return "<h1> KPMG </h1>"

@app.route('/QA', methods=['GET'])
def QA():
    if request.method == 'GET':
        query= request.args.get('input_text')
        retrieved_passages = search(query,faiss_index,total_passage, total_text, id_to_document, DPR, DPR_tokenizer, k=20)
        answer = generate_answer(retrieved_passages)
        return answer

@app.route('/summary', methods=['GET'])
def summary():
    if request.method == 'GET':
        query= request.args.get('input_text')
        retrieved_passages = search(query,faiss_index,total_passage, total_text, id_to_document, DPR, DPR_tokenizer, k=20)
        summary_txt = summarize(retrieved_passages)
        return summary_txt

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bfbd-34-122-85-123.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:34:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:34:26] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:35:03] "GET /summary?input_text=카카오의+신사업은%3F HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:35:32] "GET /QA?input_text=카카오 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:35:34] "GET /QA?input_text=카카오 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Feb/2023 20:35:42] "GET /QA?input_text=카카오의+신사업은%3F HTTP/1.1" 200 -
